In [1]:
from flask import Flask, render_template, request
from keras.preprocessing.text import Tokenizer
import tensorflow as tf
import pandas as pd
import re
import numpy as np
from sklearn.model_selection import train_test_split
from keras.preprocessing.sequence import pad_sequences
from keras import backend as K

Using TensorFlow backend.


In [2]:
def preprocess_text(sen):
    # Remove punctuations and numbers
    sentence = re.sub('[^a-zA-Z]', ' ', sen)

    # Single character removal
    sentence = re.sub(r"\s+[a-zA-Z]\s+", ' ', sentence)

    # Removing multiple spresultaces
    sentence = re.sub(r'\s+', ' ', sentence)

    return sentence

In [3]:
toxic_comments = pd.read_csv('newmaindata.csv')
filter = toxic_comments["review"] != ""
toxic_comments = toxic_comments[filter]
toxic_comments = toxic_comments.dropna()
toxic_comments_labels = toxic_comments[['rating1','rating2','rating3','rating4','rating5']]

X = []
sentences = list(toxic_comments["review"])
for sen in sentences:
    X.append(preprocess_text(sen))

y = toxic_comments_labels.values

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [4]:
# saved_model._make_predict_function()
graph = tf.get_default_graph()
sess = tf.Session()

AttributeError: module 'tensorflow' has no attribute 'get_default_graph'

In [5]:
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(x_train)

In [6]:
def predict(fdb,tokenizer):
    xp = [preprocess_text(fdb)]
    x_trainp = np.array(xp)
    x_trainp = tokenizer.texts_to_sequences(x_trainp)

    vocab_size = len(tokenizer.word_index) + 1
    maxlen = 200
    x_trainp = pad_sequences(x_trainp, padding='post', maxlen=maxlen)
    
    global sess
    global graph
    with graph.as_default():
        if 'sess' in locals() and sess is not None:
            print('Close interactive session')
            sess.close()
        K.set_session(sess)
        saved_model = tf.keras.models.load_model('new_run_cross')
        result = saved_model.predict(x_trainp)
    return result

In [7]:
from itertools import chain

In [8]:
def scorer(result):
    l = list(chain.from_iterable(result))
    ind = l.index(max(l))
    val = ''
    ind = ind+1
    if(ind==1):
        val = '128545'
    
    elif(ind==2):
        val = '128577'
    
    elif(ind==3):
        val = '128528'
    
    elif(ind==4):
        val = '128578'
    
    else:
        val = '128513'
    
    
    return ind,val

In [9]:
saved_model = tf.keras.models.load_model('nlp86')

In [10]:
app = Flask(__name__)

@app.route('/')
def show() -> 'html':
    return render_template('./page.html',the_title='feedback')

@app.route('/feedback', methods=['POST'])
def sentiment() -> 'html':
    value = request.form['ta1'] + " " + request.form['ta2'] + " " + request.form['ta3'] + " " + request.form['ta4'] + " " + request.form['g']
    prediction = predict(value,tokenizer)
#     print(prediction)
#     print(prediction.shape)
#     print(len(prediction))
    ind, val = scorer(prediction)
    return render_template('./result.html',result = prediction,index = ind,emoji = val)

app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [11/May/2020 16:01:43] "GET / HTTP/1.1" 200 -
[2020-05-11 16:02:18,657] ERROR in app: Exception on /feedback [POST]
Traceback (most recent call last):
  File "C:\Users\Shruti priya\Anaconda3\lib\site-packages\flask\app.py", line 2446, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\Shruti priya\Anaconda3\lib\site-packages\flask\app.py", line 1951, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\Shruti priya\Anaconda3\lib\site-packages\flask\app.py", line 1820, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\Shruti priya\Anaconda3\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "C:\Users\Shruti priya\Anaconda3\lib\site-packages\flask\app.py", line 1949, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\Shruti priya\Anaconda3\lib\site-packages\flask\app.py", line 1935, in

In [11]:
tf.VERSION

AttributeError: module 'tensorflow' has no attribute 'VERSION'

In [ ]:
np.set_printoptions(suppress=True)